In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization
import pickle
import os

In [ ]:
BASE_DIR = os.getcwd()
DATA_PATH = os.path.join(BASE_DIR, "stock_features.csv")
SCALER_PATH = os.path.join(BASE_DIR, "scaler.pkl")
MODEL_PATH = os.path.join(BASE_DIR, "tenson.pt")

In [ ]:
df = pd.read_csv(DATA_PATH)
df.head()

In [ ]:
features = df.columns.drop(["ticker", "label"])
X = df[features].values
y = df["label"].map({"BUY": 0, "HOLD": 1, "SELL": 2}).values

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

with open(SCALER_PATH, "wb") as f:
    pickle.dump(scaler, f)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_val = torch.tensor(y_val, dtype=torch.long)

In [ ]:
class StockClassifier(nn.Module):
    def __init__(self, input_dim=25, dropout=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(32, 3)
        )
    def forward(self, x):
        return self.net(x)

In [ ]:
def train_model(lr, dropout):
    model = StockClassifier(dropout=float(dropout))
    optimizer = torch.optim.Adam(model.parameters(), lr=float(lr))
    loss_fn = nn.CrossEntropyLoss()

    for _ in range(5):
        model.train()
        logits = model(X_train)
        loss = loss_fn(logits, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    val_logits = model(X_val)
    val_loss = loss_fn(val_logits, y_val).item()
    return -val_loss

In [ ]:
pbounds = {
    "lr": (1e-5, 1e-3),
    "dropout": (0.1, 0.5)
}

optimizer = BayesianOptimization(f=train_model, pbounds=pbounds, verbose=2)

In [ ]:
optimizer.maximize(init_points=2, n_iter=5)

In [ ]:
best_params = optimizer.max["params"]
print("Best Parameters:", best_params)

In [ ]:
final_model = StockClassifier(dropout=best_params["dropout"])
optimizer_final = torch.optim.Adam(final_model.parameters(), lr=best_params["lr"])
loss_fn = nn.CrossEntropyLoss()

for epoch in range(10):
    final_model.train()
    logits = final_model(X_train)
    loss = loss_fn(logits, y_train)
    optimizer_final.zero_grad()
    loss.backward()
    optimizer_final.step()

In [ ]:
final_model.eval()
val_logits = final_model(X_val)
val_loss = loss_fn(val_logits, y_val).item()
print(f"Validation Loss: {val_loss}")

In [ ]:
torch.save(final_model.state_dict(), MODEL_PATH)
print(f"✅ Model saved to: {MODEL_PATH}")
print(f"✅ Scaler saved to: {SCALER_PATH}")